# Setting up the regional configs at 1/36° from NACHOS12.L75 with NEMO 4.0 rev 10650

## What region

We want a geographical area that will run at 1/24° at the same cost than a full NACHOS run at 1/12° : 40° x 25° or 20° x 50° for each.

We want to cover LatMix and OSMOSIS campaign, so a Gulf Stream config and a  European coast config.

First guess :
 - GS : 82°W-35°W x 28°N-50°N
 - EU : 24.25°W-5°E x 32°N-65.5°N
 
Update on June 24th :
 - GS : 82°W-40°W x 27°N-48°N
 - EU : Iceland to Norway to Madeira Island

## The code

  - download latest version of DCM : ``` git clone https://github.com/meom-group/DCM.git DCM_4.0-10650```
  - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF, run getnemoref.sh 
  - add file 4.0-10650 in /home/albert7a/modules/NEMODRAK and add ```module load NEMODRAK/4.0-10650``` in .bashrc
  - get the modifications I added in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/RUNTOOLS/lib 
  - reference configuration /home/albert7a/CONFIGS/CONFIG_GS24.L75/GS24.L75-MAA008

## The grid

 - extract the region from eNATL36 coordinates file : /mnt/meom/MODEL_SET/eNATL36/eNATL36-I/coordinates_eNATL36.v1.nc
 
 - check if frontiers at 1/36° match a grid line in NACHOS12 ?
  
## Bathymetry

  - Jean-Marc did all the work already, bathy eNATL36 is on cal1 : /mnt/alberta/equipes/IGE/meom/MODEL_SET/eNATL36/eNATL36-I/eNATL36_bathy_gebco2014_v2.1.nc4
  - check sea mounts in GS
 
## Domain

  - use the DOMAIN_cfg tool to construct the domain from grid and bathymetry files :
    - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools
         - ```./maketools -n DOMAINcfg -m X64_OCCIGEN2jm2```
         - copy the directory into /scratch/cnt0024/hmg2840/albert7a/GS24.L75/GS24.L75-I/DOMAINcfg
         - make the change in namelist_cfg corresponding to the config (vertical grid)
         - ```./make_domain_cfg.exe```

## Run-offs

  - Jean-Marc did all the work already, where ?

## Initial conditions

  - sosie on GLORYS2V4 :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/MK_INI_GLORYS2GS24
          - GLORYS2V4 is in /store/albert7a/GLORYS2V4
          - modify tmp_namelist and tmp_namelist2D
          - ```./make_sosie.ksh```
          - ```./make_init.ksh``` (we only need january for init but the whole year for damping)
      
## BDY
  - sosie on NACHOS12-MAA4001 (2010) and MAA4002 (2011-2013) outputs :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/MK_BDY_GS24-sosie-direct
          - ```./make_bdy_target_from_mask_GS24.ksh``` to get the slices of mesh_mask corresponding to the boundaries
          - modify tmp_namelist and tmp_namelist2D
          - ```./make_sosie_NACHOS12-BDYS.ksh```
          - ```./make_monthly_and_flip.ksh``` 


## Weights for atm forcing 

  - build new weights fitted to DFS5.2 :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/WEIGHTS
          - link the files (coordinates and 1 u10 and 1 radlw )
          - ```./make_weights_dfs5_zgs24.ksh```

## Other files

 